<a href="https://colab.research.google.com/github/otvc/PythonText/blob/main/Generation_Congratulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Loading Data

In [69]:
import numpy as np

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip3 install lxml

In [7]:
!pip3 install bs4

In [11]:
!pip install requests

In [12]:
import requests

In [370]:
source = "https://pozdravok.ru/pozdravleniya/den-rozhdeniya/"
page_count = 30
text_list = []

In [371]:
def gen_url_list(url_base, count):
  gen_list = []
  for i in range(1, count + 1):
    new_url = url_base + str(i)
    gen_list.append(new_url)
  return gen_list

def get_responses(url_list):
  responses = []
  for url in url_list:
    response = requests.get(url)
    if response.status_code == 200:
      print(f'Success {url}!')
      responses.append(response)
    elif response.status_code == 404:
      print(f'Not Found. {url}')
  return responses

In [372]:
url_list = gen_url_list(source, page_count)

In [373]:
responses = get_responses(url_list)

Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/1!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/2!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/3!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/4!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/5!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/6!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/7!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/8!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/9!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/10!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/11!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/12!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/13!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/14!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/15!
Success https://pozdravok.ru/pozdravleniya/den-rozhdeniya/16!
Success https://p

In [374]:
from bs4 import BeautifulSoup

In [375]:
def get_text(responses):
  congratulations = []
  for response in responses:
    soup = BeautifulSoup(resp_test.text, 'html.parser')
    texts = soup.find_all(lambda tag: tag.has_attr('class') and tag.attrs['class'][0] == 'sfst')
    for text in texts:
      congratulations.append(text.getText(' '))
  return congratulations 

In [376]:
congrats = get_text(responses)

In [377]:
import csv

In [378]:
source_to_save = "/content/drive/MyDrive/NLP_Data/congratulations.csv"

In [379]:
ind = 0
with open(source_to_save, 'w') as csvfile:
  writer = csv.writer(csvfile, delimiter = '|')
  writer.writerow(['Id', 'Text'])
  for cng in congrats:
    writer.writerow([str(ind), cng])
    ind += 1

In [380]:
import pandas as pd

In [381]:
df = pd.read_csv(source_to_save, delimiter="|", index_col='Id')

In [382]:
df

,Text
Id,
0,С днем рождения поздравляю И от всей души жела...
1,С днем рождения поздравляю! Пожелать хочу тебе...
2,Пусть море радости и счастья Тебя повсюду окру...
3,Поздравляем с днем рождения! И желаем не груст...
4,Поздравляю с днем рождения! И желаю лишь весел...
...,...
295,"Желаю самых лучших дней, Больших мечтаний и ид..."
296,"Желаю здоровья, улыбок, добра, Безоблачных дне..."
297,"Желаю всех чудес на свете, Тепла, надежды, доб..."


#Dataset

In [383]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [384]:
import string
from collections import Counter

##Создадим Dataset и Dataloader

In [385]:
class CongratulationDataset(Dataset):
  def __init__(self, path_to_file, vector_length):
    self.words = self.load_words(path_to_file)
    self.uniq_words = self.get_uniq_words()
    self.vector_length = vector_length

    self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
    self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

    self.words_indexes = [self.word_to_index[w] for w in self.words]

  def load_words(self, path_to_file):
    df = pd.read_csv(source_to_save, delimiter="|", index_col='Id')
    text = df['Text'].str.cat(sep=' ')
    punct = string.punctuation + "…"+"»"+"«" + "—";
    trans = str.maketrans(punct, " "*len(punct))
    text = text.translate(trans).lower().split()
    return text

  def get_uniq_words(self):
    word_counts = Counter(self.words)
    return sorted(word_counts, key=word_counts.get, reverse=True)

  def __len__(self):
    return len(self.words_indexes) - self.vector_length

  def __getitem__(self, index):
    return(
        torch.tensor(self.words_indexes[index:index+self.vector_length]),
        torch.tensor(self.words_indexes[index+1:index+self.vector_length+1]),
        )

In [386]:
data_source = source_to_save

In [387]:
congratulations_data = CongratulationDataset(data_source, 64)

In [388]:
train_dataloader = DataLoader(congratulations_data, batch_size=64, shuffle=True)

In [389]:
train_features, train_labels = next(iter(train_dataloader))

##Создадим модель (LSTM в данном случае)

In [390]:
class LSTM(nn.Module):
  def __init__(self, dataset, input_size, emb_dim, num_layers):
    super(LSTM, self).__init__()

    self.input_size = input_size
    self.embedding_dim = emb_dim
    self.num_layers = num_layers

    self.lstm = nn.LSTM(
        input_size = self.input_size,
        hidden_size = self.input_size,
        num_layers = self.num_layers,
        dropout = 0.2,
    )

    dataset_size = len(dataset.uniq_words)

    self.embedding = nn.Embedding(dataset_size, self.embedding_dim)
    self.fc = nn.Linear(self.input_size, dataset_size)

  def forward(self, x, prev_state):
    x = self.embedding(x)
    x, s = self.lstm(x, prev_state)
    logit = self.fc(x)
    return logit, s
    
  def init_state(self, sequence_length):
      return (torch.zeros(self.num_layers, sequence_length, self.input_size),
              torch.zeros(self.num_layers, sequence_length, self.input_size))

In [391]:
model = LSTM(congratulations_data, 64, 64, 2)

##Тренировка

In [392]:
def train_loop(model, dataloader, loss_fn, optimizer, epochs, sequence_length):
  for epoch in range(1, epochs + 1):
    h, c = model.init_state(sequence_length)
    for batch, (X, y) in enumerate(dataloader):
      optimizer.zero_grad()
      
      y_pred, (h, c) = model(X, (h, c))
      
      loss = loss_fn(y_pred.transpose(1, 2), y)

      h = h.detach()
      c = c.detach()

      loss.backward()
      optimizer.step()
      #print(f"Batch: {64 * batch} -- loss: {loss}")
    print(f"Epoch: {epoch} -- loss: {loss}")

In [393]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

In [394]:
train_loop(model, train_dataloader, loss, optimizer, 20, 64)

Epoch: 1 -- loss: 2.5594098567962646
Epoch: 2 -- loss: 1.1407605409622192
Epoch: 3 -- loss: 0.9620330929756165
Epoch: 4 -- loss: 1.0006530284881592
Epoch: 5 -- loss: 0.9455973505973816
Epoch: 6 -- loss: 0.905251681804657
Epoch: 7 -- loss: 0.8708588480949402
Epoch: 8 -- loss: 0.9185413122177124
Epoch: 9 -- loss: 0.853701114654541
Epoch: 10 -- loss: 0.87147456407547
Epoch: 11 -- loss: 0.9034827947616577
Epoch: 12 -- loss: 0.8142960667610168
Epoch: 13 -- loss: 0.8521782159805298
Epoch: 14 -- loss: 0.8378504514694214
Epoch: 15 -- loss: 0.87010657787323
Epoch: 16 -- loss: 0.8435150384902954
Epoch: 17 -- loss: 0.9344965815544128
Epoch: 18 -- loss: 0.9472817182540894
Epoch: 19 -- loss: 1.004370927810669
Epoch: 20 -- loss: 0.873436450958252


In [395]:
def predict(dataset, model, text, next_words=100):
    model.eval()

    words = text.split(' ')
    state_h, state_c = model.init_state(len(words))

    for i in range(0, next_words):
        x = torch.tensor([[dataset.word_to_index[w] for w in words[i:]]])
        y_pred, (state_h, state_c) = model(x, (state_h, state_c))

        last_word_logits = y_pred[0][-1]
        p = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=p)
        words.append(dataset.index_to_word[word_index])
    return ' '.join(words)

In [396]:
output = predict(congratulations_data, model, text='счастья')

In [397]:
'с ' + output

'с счастья тебя повсюду окружит обходят стороной ненастья и желаем не забывается а в семье благополучия друзья чтоб были чтоб активы жизнь ворвутся её везением озарят помогут к ней для счастливых событий в душе всегда достатка с днем рождения и вновь и мечты твои веселее желаю быть всегда на долгие года с тобою будет пусть море радости добра безоблачных дней и хороших новостей пусть друзья во всех чудес на высоте жить счастливо и хороших новостей пусть в день не только в банке были лучшие во всех чудес на свете тепла надежды и длились все будни твои поздравляем с тобою будет пусть здоровье'